# Compare Models

Create object to automated model comparisons

In [7]:
import arithmetic_datasets as ad

from tensorflow.keras.layers import Input, Lambda, Dense, concatenate
from tensorflow.keras.models import Model

import matplotlib.pyplot as plt
import random
import numpy as np

from tensorflow.keras.callbacks import EarlyStopping

import numpy as np
import tensorflow as tf
import random
import keras
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.metrics import Metric

### ModelComparator

```
mc = ModelComparator()
mc.add_model(name, model)
mc.add_training_set(name, data)

mc.train_models(epochs, batches, **kwargs)

mc.compare_min(metric)
mc.compare_max(metric)

mc.history_for(model_name, training_set_name)
```

In [8]:
class ModelComparator:
    def __init__(self):
        self.models = {}
        self.training_sets = {}
        self.histories = {}
    
    def add_model(self, name, model):
        self.models[name] = model
    
    def add_training_set(self, name, data):
        self.training_sets[name] = data
        
    def train_models(self, **kwargs):
        for model_name in self.models:
            for tset_name in self.training_sets:
                print(f"MODEL {model_name} TRAINING ON DATASET {tset_name}")
                
                model = self.models[model_name]()
                
                history = model.fit(
                    self.training_sets[tset_name][0], self.training_sets[tset_name][1],
                    **kwargs
                )
                
                self.histories[(model_name, tset_name)] = history
    
    def compare_min(self, metric):
        result = {}
        minimum = None
        min_name = None
        for hkey in self.histories:
            history = self.histories[hkey].history
            result[hkey] = np.amin(history[metric])
            if minimum == None or minimum >= result[hkey]:
                minimum = result[hkey]
                min_name = hkey
        return result, {"name" : min_name, "value":minimum}
            
    
    def compare_max(self, metric):
        result = {}
        maximum = None
        max_name = None
        for hkey in self.histories:
            history = self.histories[hkey].history
            result[hkey] = np.amax(history[metric])
            if maximum == None or maximum >= result[hkey]:
                maximum = result[hkey]
                max_name = hkey
        return result, {"name" : max_name, "value": maximum}
    
    def history_for(self, model_name, training_set_name):
        return self.histories[(model_name, training_set_name)]

### Create Models

In [9]:
def nondense_model():
    # Input layer of 3 neurons 
    inp = Input(shape=(1,3))
    
    #128 layer
    d2_out = Dense(128)(inp)

    #grab first, 2nd half of the 128 layer
    d2_out_p1 = Lambda(lambda x: x[:,:,0:64])(d2_out)
    d2_out_p2 = Lambda(lambda x: x[:,:,64:128])(d2_out)

    #64 layer(s)
    d3_out = Dense(64)(d2_out_p1)
    d4_out = Dense(64)(d2_out_p2)

    #grab output nodes from both 64 layers
    d5_out = concatenate([d3_out, d4_out])
    
    o = Dense(1)(d5_out)
    
    model = Model(inp, o)

    model.compile(
        loss="MeanSquaredError",
        metrics=['accuracy']
    )
    
    return model

def dense_model_5L():
    model_5layer = tf.keras.models.Sequential([
        tf.keras.layers.Dense(1024, input_shape=(1,3)),
        tf.keras.layers.Dense(512),
        tf.keras.layers.Dense(256),
        tf.keras.layers.Dense(128),
        tf.keras.layers.Dense(64),
        tf.keras.layers.Dense(1)
    ])
    
    model_5layer.compile(
        loss="MeanSquaredError",
        metrics=['accuracy'] #Acc not working, in testing
    )

    return model_5layer

### Datasets

In [10]:
num_examples = 2000000
rstart = -50000
rend = 50000

add_setX, add_setY = ad.gen_data_add(num_examples, rstart, rend)
sub_setX, sub_setY = ad.gen_data_sub(num_examples, rstart, rend)

add_setX = add_setX.reshape(num_examples, 1, 3)
sub_setX = sub_setX.reshape(num_examples, 1, 3)

# 50-50
step = int(0.5*num_examples)

addsub_set1X = np.concatenate((add_setX[:step], sub_setX[step:]))
addsub_set1Y = np.concatenate((add_setY[:step], sub_setY[step:]))

# 80-20
step = int(0.8*num_examples)
addsub_set2X = np.concatenate((add_setX[:step], sub_setX[step:]))
addsub_set2Y = np.concatenate((add_setY[:step], sub_setY[step:]))

### Build ModelComparator

In [11]:
mc = ModelComparator()

mc.add_model("nondense_model", nondense_model)
mc.add_model("dense_model", dense_model_5L)

mc.add_training_set("addition", (add_setX, add_setY))
mc.add_training_set("subtraction", (sub_setX, sub_setY))
mc.add_training_set("add-sub 50/50", (addsub_set1X, addsub_set1Y))
mc.add_training_set("add-sub 80/20", (addsub_set2X, addsub_set2Y))

### Train Models

In [ ]:
num_epochs = 100
batch_size = 50000

mc.train_models(
    batch_size=batch_size,
    epochs=num_epochs
)

MODEL nondense_model TRAINING ON DATASET addition
Epoch 1/100
40/40 [==============================] - 9s 215ms/step - loss: 42905188.0000 - accuracy: 9.0000e-06
Epoch 2/100
40/40 [==============================] - 9s 226ms/step - loss: 4189947.5000 - accuracy: 1.0500e-05
Epoch 3/100
40/40 [==============================] - 8s 209ms/step - loss: 4059270.7500 - accuracy: 1.0000e-05
Epoch 4/100
40/40 [==============================] - 8s 211ms/step - loss: 3908683.0000 - accuracy: 9.5000e-06
Epoch 5/100
40/40 [==============================] - 9s 222ms/step - loss: 3705618.2500 - accuracy: 1.0500e-05
Epoch 6/100
40/40 [==============================] - 9s 214ms/step - loss: 3474075.2500 - accuracy: 7.0000e-060s - loss: 3474075.2500 - accuracy: 7.0000e-
Epoch 7/100
40/40 [==============================] - 7s 173ms/step - loss: 3241212.0000 - accuracy: 1.0500e-05
Epoch 8/100
40/40 [==============================] - 7s 171ms/step - loss: 3047705.0000 - accuracy: 9.0000e-06
Epoch 9/100
40/40

40/40 [==============================] - 6s 160ms/step - loss: 616600.9375 - accuracy: 7.5000e-06
Epoch 73/100
40/40 [==============================] - 7s 163ms/step - loss: 619719.5625 - accuracy: 9.5000e-06 2s - loss: 615036.5625 - 
Epoch 74/100
40/40 [==============================] - 7s 163ms/step - loss: 598483.8125 - accuracy: 1.1000e-05
Epoch 75/100
40/40 [==============================] - 6s 157ms/step - loss: 601518.9375 - accuracy: 1.1000e-05 0s - loss: 608879.6250 - accuracy: 1.0
Epoch 76/100
40/40 [==============================] - 7s 164ms/step - loss: 614132.8125 - accuracy: 7.5000e-06
Epoch 77/100
40/40 [==============================] - 6s 152ms/step - loss: 578794.1875 - accuracy: 7.5000e-06
Epoch 78/100
40/40 [==============================] - 7s 163ms/step - loss: 583742.5625 - accuracy: 7.0000e-06 0s - loss: 588220.1875 - accuracy: 7.6
Epoch 79/100
40/40 [==============================] - 6s 160ms/step - loss: 594267.2500 - accuracy: 1.0500e-05
Epoch 80/100
40/40 [=

### compare min loss

In [ ]:
models_min_loss, min_loss = mc.compare_min("loss")

In [ ]:
min_loss

In [ ]:
models_min_loss